Geologic Image Processing in Python
===============================

As a geoscientist, some of the most useful and frequently-used computational tools fall under the broad category of image processing.  It's more than working with photographs or satellite imagery, though.  All "image processing" means in this context is working with data that's on a regular grid. For example, a digital elevation model is every bit as much an image as a core photograph is. 

Outline for Today
--------------------------

  * Overview / Introduction
  * Seamount Detection Example
    - Thresholding
    - Filtering
    - Segmentation
    - Simplification
  * Slope and Hillshade of Topographic Data
    - Gradients
  * Grain Detection in Thin Sections
    - Using gradients for more than slope
  * Lineament Analysis from Aerial Photography
    - Edge detection
    - Hough Transform
    - Structure Tensor
  * What is color?
    - Satellite data examples
    
  

#### Goals

This tutorial will introduce you to some core image processing methods by solving a handful of realistic tasks related to geology and geophysics.  The goal is to gain familiarity with key "building blocks" and terminology so that you can understand how to use common Python libraries such as `scipy.ndimage` and `sklearn` in your day-to-day work.  For many of you, these may seem like simple tasks and things that are trivial to accomplish in ArcGIS, ImageJ, or Photoshop.  However, the terminology is a bit different when working with image processing and computer vision libraries.  Many operations are called very different things, or are broken into smaller pieces.  Therefore, it's important to understand how to string the fundamental operations that are usually exposed in programming libraries into the higher-level operations you're used to thinking about.  

There are a lot of great tutorials out there already for the libraries we'll be working with.  However, there are not many geoscience-focused examples freely available.  It's much easier to see how methods can be applied to your domain when there are examples of familiar tasks related to what you're doing.  

Often, particularly with image processing, what we want to do as scientisits is significantly different than the tasks that image processing tutorials are aimed at.  The methods are the same, but thousands of examples of manipulating cat photos doesn't always help form a connection to an analysis you're stuck on.  Hopefully this tutorial can provide a bit of a bridge between the two worlds.

This is not meant to be a complete introduction to image processing, or even a complete introduction to common geoscience image processing problems. Those of you already familiar with image processing methods will notice that we completely gloss over some very important points and do not fully explain many underlying methods.  The goal here, however, is to give you a quick overview of what's possible by combinging a few well-known methods.  Hopefully after this tutorial you feel comfortable enough to start experimenting and learning more on your own.

#### Libraries

We'll focus on using a combination of `numpy`, `scipy.ndimage`, and `sklearn`. `sklearn` is a leading image processing library that has many very nice features and exposes many advanced methods.  `scipy.ndimage` is a bit more low-level, but has the advantage of both working in 3D (or N-D) and focusing on efficient implementations of common operations.   We'll also use libraries like `rasterio` for reading and writing geospatial data, but we won't dive deeply into the details of working with geospatial data.

#### Notes

Because the Transform2020 tutorial is remote, it's difficult to provide the type of hands-on help we normally would.  Therefore, this set of notebooks is meant to be a "cookbook" demonstrating common tasks and illustrating underlying principles through specific examples.  We won't go over all of the details, but hopefully you can come back to these examples later and adjust them to your needs.

#### Additional Resources

This tutorial assumes basic Python knowledge and at least some familiarity with `numpy`.  However, if you're new to all of this, that's perfectly okay!  In that case, focus on how the operations are chained together more than the details of the code. If you're looking for tutorials on learning Python for scientific purposes, here are some resources you may find useful:

Scipy-lectures is a great introduction to scientific computing in python: https://scipy-lectures.org/index.html  I can't recommend this strongly enough, honestly.  If you've picked up basic python syntax and looking for a place to start actually applying python, this is a great practical guide to the tools you'll need to know most.  It has sections on most of the libraries we'll use today, and I'd recommend browsing through it even if you're an expert.

The (new) official `scipy.ndimage` tutorial is also a good overview of a library we'll be using extensively here: https://docs.scipy.org/doc/scipy/reference/tutorial/ndimage.html  However, it's most useful if you're already familiar with the basics of both image processing and numpy/scipy operations.  You may find it a bit dense otherwise, but it's full of excellent examples of all of the key functionality in `scipy.ndimage`.

There are many `skimage` tutorials out there, but the gallery is a great place to start: https://scikit-image.org/docs/dev/auto_examples/index.html  Image processing operations are often visual, so it's not uncommon to suspect something is easily accomplished but not know the name of the operation.  The `skimage` gallery is something of a visual index to many of the operations in the library.

Diving in: Identifying Seamounts
------------------------------------------------

Let's get started with some concrete examples.  If you'd like to see where we're going, you can jump straight to the complete example and run it:

In [ ]:
%matplotlib notebook
%load examples/seamount_detection.py

We're going to try to detect, count, and calculate areas of seamounts based on bathymetry data.  Along the way, we'll cover the following image processing concepts:

  * Array representation
  * Thresholding
  * Filtering
  * Segmentation
  
Let's start by loading our data from a geotiff and taking a look at it:

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio

from context import data

# Let's load data from a geotiff using rasterio...
with rio.open(data.gebco.seamounts, 'r') as src:
    bathy = src.read(1)
    
print(bathy)

So `bathy` is a 2D array with integer values.  The units are meters relative to sea level.  Note that more or less everything is negative: This is GEBCO bathymetry data from a of the Western Pacific near the Marianas Trench.

Let's take a look at what this data looks like:

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(bathy, cmap='gray')
fig.colorbar(im, orientation='horizontal')
plt.show()

We have a single array of data that we're displaying as grayscale.  Let's go ahead and add some color to that. We'll discuss color in images in more detail later, but this is a good chance to briefly introduce using colormaps in matplotlib:

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(bathy, cmap='Blues_r', vmax=0)
im.cmap.set_over('green') # Just display any land as green...
fig.colorbar(im, orientation='horizontal')
plt.show()

Okay, so we're looking at a large number of seamounts rising up above the abyssal plain.  It's really obvious where they are visually, but it would be nice to be able to quickly identify them programatically.  For example, we might want to look at their distribution by area or volume, or to just get a count without manually counting all of them.

### Thresholding

The simplest approach we could take would be to threshold the bathymetry data.  The abyssal plain is usually around 4km depth due to the relatively constant thickness and density of oceanic crust.  Therefore, we could try thresholding out anything above 3500 meters.  Because this is a numpy array, the operation is quite simple:

In [ ]:
simple_threshold = bathy > -3500
print(simple_threshold)

Note that we have an array of True/False values. This is a boolean array, and some of the operations we'll work with today only operate on these sort of boolean True/False arrays. 

Often, in image processing, you'll convert the True/False representation into a 1/0 representation.  Behind the scenes, the `True` values above can be efficiently converted into `1` and the `False` values into `0`. For example:

In [ ]:
# "view" only changes the way we're interepting the underlying data.
# If you're not familiar with "view" vs "astype", use "astype". All I'm
# showing here is that it's seamless to go from True/False --> 1/0
print(simple_threshold.view(np.uint8)) 

Okay, enough beating around the bush. Let's take a look at what we've accomplished:

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, sharey=True)
axes[0].imshow(bathy, cmap='Blues_r', vmax=0)
axes[1].imshow(simple_threshold.view(np.uint8))

for ax in axes.flat:
    ax.set(xticks=[], yticks=[])
fig.tight_layout()

plt.show()

The yellow regions are `True` in the boolean array. Note that we capture many seamounts out in the abyssal plain, but classify the entire volcanic arc and forearc in the west as a single large seamount.  We also miss a lot of smaller seamounts that 

Let's take a second to make a fancier display so we can explore what we capture and what we don't.  I'm going to use a quick utility included with this tutorial to allow toggling of different overlays on the plot.  We'll re-use this throughout the tutorial.

In [ ]:
from context import utils

fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green') 

# We'll mask any False values so that they're transparent
im = ax.imshow(np.ma.masked_where(~simple_threshold, simple_threshold),
               vmin=0, vmax=1, label='>3500 mbsl')

ax.set(xticks=[], yticks=[])
fig.tight_layout()

utils.Toggler(im).show()

As you can see, we're doing an okay job of detecting large seamounts, but an awful job of detecting smaller ones and arc volcanoes.  This is because we're using a fixed elevation threshold to determine whether or not something is a seamount.

Visually, we'd determine whether or not a pixel is part of a seamount based on the area around it. We're looking for features that rise up from the surrounding topography.  However, that "base level" of topography varies throughout our study area.  Therefore we need a way of finding the "background" elevation.  Remember that -- we'll come back to it soon.

### Filters

Filters (and convolution, which is verly closely related) are an ubiquitous concept in image processing.  Filtering an image is a type of "moving window" operation.  For each pixel in the image, we take a region around it and apply some operation based on that region to define a new pixel value.  Most commonly used filters involve multiplying each pixel in the region by a weight and then summing (i.e. a convoluion). The array of weights is usually [called a kernel](https://en.wikipedia.org/wiki/Kernel_(image_processing)). This allows blurring, sharpening, edge detection, and many other useful operations.  

Other filers aren't defined by weights, but by more flexible operations.  A simple example of this is a median filter, where the value of the pixel is the median of the pixels in some window surrounding it.  To calculate a median, we need to sort all the pixels we're using and find the one in the middle -- it can't be defined by a weighted average.  As a result, it can become slow if a large window is used.  (Note: in practice, there are some shortcuts to the "full" sort, but either way a median filter is slower than filters defined by weights.)

Let's use one of the simplest possible filters: [a uniform filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.uniform_filter.html).  A uniform filter is an average of all pixel values in a square region.  It's simple, but it's fast.  In practice, it blurs the result:

In [ ]:
import scipy.ndimage

fig, ax = plt.subplots()
im = ax.imshow(bathy, cmap='Blues_r', vmax=0)

ax.set(xticks=[], yticks=[])

# Values are width of the square window in _pixels_
def update(value):
    blurred = scipy.ndimage.uniform_filter(bathy, value)
    im.set_data(blurred)

utils.Slider(ax, 0, 150, update, start=50).show()

Okay, so we mentioned this was a moving window. We need to think about what happens at the edges.  By default, most functions in `scipy.ndimage` use "reflect" boundary conditions.  That's perfect for this use case, but it's good to have a look at the other options.  The `mode` kwarg controls how boundaries are handled: (this is the same for any function where boundaries matter in `scipy.ndimage`)

In [ ]:
scipy.ndimage.uniform_filter?

### Using filters in seamount detection

Going back to our seamount detection problem, let's use the blurred / uniform-filtered bathymetry to define the "background" elevation.  Seamounts or other peak-like features will be significantly higher than the background elevation and trenches or other trough-like features will be significantly below it.  Therefore, we can identify seamounts by comparing the uniform-filtered bathymetry data to the original bathymetry data.  Anything that's more than some amount higher than the background elevation, we'll consider a seamount:

In [ ]:
# We'll use a simple filter to define the local background elevation and
# assume anything more than 500m above the background is a seamount
blurred = scipy.ndimage.uniform_filter(bathy, 150)
better_threshold = bathy > (blurred + 500)

fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green') 

# Compare this to our original 3500m threshold
im1 = ax.imshow(np.ma.masked_where(~simple_threshold, simple_threshold),
                vmin=0, vmax=1, label='>3500m')
im2 = ax.imshow(np.ma.masked_where(~better_threshold, better_threshold),
                vmin=0, vmax=1, label='Above Background')

ax.set(xticks=[], yticks=[])
fig.tight_layout()

utils.Toggler(im1, im2).show()

### Cleanup of Thresholded Regions

As you can see, we've done a better job identifying smaller seamounts, and we're no longer identifying the entire volcanic arc as one gigantic seamount.

However, we're also identifying some very tiny features that we'd rather leave out, and the boundaries of each feature are very rough.   It would be nice to "clean up" our classification so that we have smoother boundaries, holes are filled in, and very small regions are excluded.

To do this, we'll operate on our thresholded boolean array, rather than operating on the bathymetry data directly.  

First, let's fill any "holes" in our classified areas.  Anything that's fully surrounded by something we're considering a seamount is clearly a seamount as well.  There aren't many holes in our classification, but if you look around, you can find several small ones.  They may not matter much for this case, but it's an extremely common post-processing step in classifications, and one that's very frequently handy to know how to do.

To fill holes, we'll rely on [mathematical morphology](https://en.wikipedia.org/wiki/Mathematical_morphology) (which is a cornerstone of image processing that comes from geology, by the way).  Most mathematical morphology operators work on boolean arrays similar to what we have.  It provides ways of identifying connected and non-connected regions, buffering, eroding, and similar key operations on classified images.  It's straight-forward to identify holes that are surrounded by True values in a boolean array, and `scipy.ndimage` even gives us a one-step function to do it:  [`scipy.ndimage.binary_fill_holes`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.binary_fill_holes.html#scipy.ndimage.binary_fill_holes)

In [ ]:
# Filling holes in our classified regions is straight-forward
filled = scipy.ndimage.binary_fill_holes(better_threshold)

# And let's compare the results:
fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green') 

im1 = ax.imshow(np.ma.masked_where(~better_threshold, better_threshold),
                vmin=0, vmax=1, label='Original')
im2 = ax.imshow(np.ma.masked_where(~filled, filled),
                vmin=0, vmax=1, label='Filled')

ax.set(xticks=[], yticks=[])
fig.tight_layout()

utils.Toggler(im1, im2).show()

Next, let's do something a bit more interesting.  Let's clean up our classification with a single filter.  This will smooth boundaries and eliminate very small features we're not interested in.

Remember the median filter we very briefly talked about earlier?  You can also apply a median filter to boolean True/False data.  In that case, the pixel will be whatever the majority of pixels in the region around it are.  The large a region (a.k.a. kernel) we choose, the smoother/simpler the result will be.  

Let's have a look at that and experiment with different filter sizes. We'll use [`scipy.ndimage.median_filter`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.median_filter.html#scipy.ndimage.median_filter) for this.

Quick note: If we had more than one class (e.g. say this was a land use classification with water, urban, forest, and agriculture classes), the equivalent "cleanup" operator would be a [majority filter](https://scikit-image.org/docs/stable/api/skimage.filters.rank.html#majority).  For a boolean True/False array, a median filter is equivalent to a majority filter.

This will take a bit to run. A median filter is a much slower operation than the other filters we've applied so far...

In [ ]:
widths = [3, 5, 11, 21]
versions = ([filled] +
            [scipy.ndimage.median_filter(filled, x) for x in widths])

fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0)

ims = []
for width, version in zip([0] + widths, versions):
    transparent = np.ma.masked_equal(version.view(np.uint8), 0)
    im = ax.imshow(transparent, vmin=0, vmax=1, label=f"{width}x{width}")
    ims.append(im)

ax.set(xticks=[], yticks=[])
fig.tight_layout()

utils.Toggler(*ims).show()

Let's choose a final width and we'll fill holes again, as the median filter can introduce _new_ holes into the result:

In [ ]:
cleaned = scipy.ndimage.median_filter(better_threshold, 13)
cleaned = scipy.ndimage.binary_fill_holes(cleaned)

Okay, so we've simplified things reasonably well, but we still have a single True/False array.  

### Identifying individual features

Originally we said we were going to count the seamounts and calculate areas.  How do we go from a bunch of True/False regions to a count?  The answer is to use mathematical morphology again.  We can identify and label each distinct group of `True` pixels that touch each other but do not touch any other `True` pixels.  In `scipy.ndimage`, this is the [`label`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.label.html#scipy.ndimage.label) function.  It assigns a unique value to each group of pixels that touch each other:

In [ ]:
labels, count = scipy.ndimage.label(cleaned)

fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0)
ax.imshow(np.ma.masked_equal(labels, 0), cmap='tab20b')

ax.set(xticks=[], yticks=[],
       title=f'There are {count} different seamounts!')
fig.tight_layout()
plt.show()

Okay, great! `labels` is now an array where each separate region has a unqiue value. The colormap doesn't quite show it, but if you hover over each seamount, you'll see that the value is different.

Remember we mentioned calculating area?  Let's go ahead and get a pixel count for each seamount.  We could do something like:

```
seamount_area = []
for i in range(1, count + 1):
    seamount_area.append((labels == i).sum())
```

However, that's a bit inefficient in this case, and `scipy.ndimage` has a builtin function to do similar "zonal statistics", so let's go ahead and use [`scipy.ndimage.sum`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.sum.html) to get a pixel count for each individual seamount and then plot a histogram:

In [ ]:
pixel_counts = scipy.ndimage.sum(cleaned, labels, np.arange(count)+1)

fig, ax = plt.subplots()
ax.hist(pixel_counts, bins=40)
ax.set(ylabel='Number of seamounts', xlabel='Area in pixels')
plt.show()

Now we have a histogram, but what the heck does "area in pixels" mean?  Units matter!

### Calculating Area (for a geographic WGS84 raster)

Let's convert that to something more sensible.  We're working with data projected in geographic WGS84 in this case.  We can find the cellsize of the pixels, but it's in degrees.  Because it's in degrees, the area of the pixel varies by latitude.  

Normally, we'd just multiply by a constant factor to convert pixel counts to area, but because we're working in degrees, the area varies depending on location.

We can very closely approximate the true area with a simple conversion.  A degree of latitude is a constant ~111.3 km. A degree of longitude is ~111.3 km at the equator. At other latitudes, multiply by the cosine of the latitude to get the width of a degree of longitude.  If you need more precision, you can reproject into a proper projection, but this is precise enough for most purposes.

Let's use `rasterio` to get the latitude and longitude of each pixel and use that to calculate the area of each pixel, then we'll sum over each seamount's region to get their total area.

In [ ]:
# Calculate area for each pixel (memory-inefficient, but fine for now)
i, j = np.mgrid[:bathy.shape[0], :bathy.shape[1]]
with rio.open(data.gebco.seamounts, 'r') as src:
    cellsize = src.transform.a # Only because pixels are square an north-south
    lon, lat = src.xy(j, i)
area = (cellsize * 111.3)**2 * np.cos(np.radians(lat))

areas = scipy.ndimage.sum(area, labels, np.arange(count)+1)

fig, ax = plt.subplots()
ax.hist(areas, bins=40)
ax.set(ylabel='Number of seamounts', xlabel='Area in $km^2$')
plt.show()

Quick aside... Note the shape of the histogram.  It's very similar to a [log-normal distribution](https://en.wikipedia.org/wiki/Log-normal_distribution)  (i.e. see the long tail on the right hand side).  This is very common in any natural process that involves areas or volumes.  As an arm-wavy explanation: If you have many independent normally distributed variables (say, width and height) and you multiply them together, the resulting distribution will be approximately log-normal.  (We don't actually expect seamount size to be as simple as width x height in this case, but still, expect log-normal distributions instead of normal anytime you start working with areas or volumes.)  Be careful about the way you interpet statistics like standard deviation or mode anytime area or volume pops up.

### But I Need to Use This Data Elsewhere...

I don't want to spend too long talking about geopatial data formats and I/O.  However, it's the natural next step to ask.  Therefore, I've included an example that both saves what we've done as a geotiff and vectorizes our regions and saves them as a shapefile:

In [ ]:
%load examples/seamount_detection_saving.py

### Review

Okay, let's quickly review the steps we took and have a look at a complete, stand-alone example.  To detect seamounts, we did the following:

  1. Load bathymetry data into an array
  2. Tried thresholding based on a constant elevation (didn't work well)
  3. Estimated background elevations with a local average (uniform filter)
  4. Detected seamounts as being regions more than 500m above the background (worked well)
  5. Cleaned up our detection to remove small regions and have smoother boundaries (median filter)
  6. Filled in any holes in our detected seamounts
  7. Separated each connected region of pixels into a unique seamount (i.e. count how many)
  8. Calculated the area distribution of seamounts

In [ ]:
%load examples/seamount_detection.py

Take-home Question:
-------------------------------

If you're comfortable with what we've done so far, here's a short challenge for you to try later:

Some of these seamounts the surface and form islands.  We usually don't consider a feature to be a seamount if a part of it reaches the surface.  **Can you exclude all seamounts that reach the surface?**

(Hint: This mostly needs numpy boolean indexing.  There's also another zonal statistics function in [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html#measurements) that you might find useful, but there's definitely more than one way to accomlish this.)

If you're very familiar with numpy alread and that seems too obvious, try some of the [mathematical morpohology operations](https://docs.scipy.org/doc/scipy/reference/ndimage.html#morphology) we didn't talk about. How would you join together seamounts that are within some distance of X pixels of each other so that they're considered a single connected region? (Hint: The most efficient way will change the shape of the regions slightly. There's more than one way to approach it, though.)

There are example answers in the `examples` folder for both of these if you get stuck. However, there's very much more than one right way to do it, so I'd encourage experimentation!

Break!
----------

Let's take five minutes. My voice needs a break!

Next Section
------------------

We'll move on to talking about gradients in images next. We'll start out with simple slope and hillshade calculations, but then use the same methods to identify separate grains in thin section images.

[02 - Gradients and Edges](./02%20-%20Gradients%20and%20Edges.ipynb)